In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
import torch

# ---------- 2. Load models ----------
# Model for semantic similarity
sim_model = SentenceTransformer("microsoft/codebert-base")

# Commit predictor model (for rectification)
pred_model_name = "mamiksik/CommitPredictorT5"
pred_tokenizer = AutoTokenizer.from_pretrained(pred_model_name)
pred_model = AutoModelForSeq2SeqLM.from_pretrained(pred_model_name)

No sentence-transformers model found with name microsoft/codebert-base. Creating a new one with mean pooling.
c:\Users\Surrya Gokul\anaconda3\envs\genai\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Surrya Gokul\.cache\huggingface\hub\models--microsoft--codebert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
from tqdm import tqdm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pred_model = pred_model.to(device)

def batch_rectify_messages(diffs, batch_size=16):
    cleaned = [clean_diff(d) for d in diffs]
    results = []

    # tqdm gives ETA automatically
    for i in tqdm(range(0, len(cleaned), batch_size), desc="Generating commits"):
        batch = cleaned[i:i+batch_size]
        inputs = pred_tokenizer(
            batch,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        ).to(device)

        with torch.no_grad():
            outputs = pred_model.generate(**inputs, max_length=64)

        decoded = pred_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        results.extend(decoded)

    return results

df["LLM Inference"] = batch_rectify_messages(df["Diff"].tolist(), batch_size=16)


Generating commits: 100%|██████████| 979/979 [1:51:09<00:00,  6.81s/it]  


In [ ]:
df.to_csv("llm_inference.csv", index=False)

In [1]:
import pandas as pd
df = pd.read_csv("llm_inference.csv")

In [2]:
df.head()

,Hash,Message,Hashes of parents,Is a merge commit?,Filename,Source Code (before),Source Code (current),Diff,LLM Inference
0,d2fdcba29db2a88056cfa87d9abfbdde840bd8c2,fix test name (#22),21b10ffb130aeac3131899e551939338b0dcebef,No,test_openai.py,"""""""Test OpenAI API wrapper.""""""\n\nfrom langcha...","""""""Test OpenAI API wrapper.""""""\n\nfrom langcha...","@@ -3,8 +3,8 @@\n from langchain.llms.openai i...",add missing docstring
1,6a3dca888ba75e219185c54cd0f0ba72d1f90a26,Fix cohere integration (#33)\n\nCurrently the ...,c7f9c62532f8137db2c4ab80335ac1472e79d56a,No,cohere.py,"""""""Wrapper around Cohere APIs.""""""\nimport os\n...","""""""Wrapper around Cohere APIs.""""""\nimport os\n...","@@ -22,7 +22,7 @@ class Cohere(BaseModel, LLM)...",model attribute for gptd-instruct-tft
2,90a6e578bc8d2147f5fcc22d6413686027eca2b5,fix type hint (#34),6a3dca888ba75e219185c54cd0f0ba72d1f90a26,No,cohere.py,"""""""Wrapper around Cohere APIs.""""""\nimport os\n...","""""""Wrapper around Cohere APIs.""""""\nimport os\n...","@@ -22,7 +22,7 @@ class Cohere(BaseModel, LLM)...",model attribute for model_name_to_json
3,eb36317f9ae6054a9bbf62ac7ceb8bae388ee811,Harrison/fix imports (#72)\n\nfix imports and ...,a5b61d59e193c4f9804c9dece7bd1a1c5c1103e0,No,embeddings.ipynb,"{\n ""cells"": [\n {\n ""cell_type"": ""code"",\n...","{\n ""cells"": [\n {\n ""cell_type"": ""code"",\n...","@@ -5,11 +5,26 @@\n ""execution_count"": 1,\n...",update error message
4,eb36317f9ae6054a9bbf62ac7ceb8bae388ee811,Harrison/fix imports (#72)\n\nfix imports and ...,a5b61d59e193c4f9804c9dece7bd1a1c5c1103e0,No,mrkl.ipynb,"{\n ""cells"": [\n {\n ""cell_type"": ""code"",\n...","{\n ""cells"": [\n {\n ""cell_type"": ""markdown...","@@ -1,5 +1,14 @@\n {\n ""cells"": [\n+ {\n+ ...",add a notebook to the default cell_type


In [3]:
curr = df.iloc[:1000]

In [4]:
curr["Diff"] = curr["Diff"].astype(str)
curr["Message"] = curr["Message"].astype(str)
curr["LLM Inference"] = curr["LLM Inference"].astype(str)

C:\Users\Surrya Gokul\AppData\Local\Temp\ipykernel_12596\944174093.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr["Diff"] = curr["Diff"].astype(str)
C:\Users\Surrya Gokul\AppData\Local\Temp\ipykernel_12596\944174093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr["Message"] = curr["Message"].astype(str)
C:\Users\Surrya Gokul\AppData\Local\Temp\ipykernel_12596\944174093.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [5]:
import pandas as pd
import re
from collections import Counter
from typing import Set, Tuple, List, Dict

# Regex to find identifiers (like variable names)
IDENT_RE = re.compile(r"[A-Za-z_][A-Za-z0-9_]{2,60}")

def split_identifier_tokens(identifier: str) -> List[str]:
    if not identifier:
        return []
    s = re.sub(r'[^0-9A-Za-z]+', '_', identifier.strip())  
    parts = [p for p in s.split('_') if p]  
    tokens = []
    for p in parts:
        # Try to split camelCase or numbers
        camel_split = re.findall(r'[A-Z]?[a-z]+|[A-Z]+(?![a-z])|\d+', p)
        if camel_split:
            tokens.extend([t.lower() for t in camel_split])
        else:
            tokens.append(p.lower())
    return [t for t in tokens if len(t) > 1] 

def extract_identifiers(text: str) -> List[str]:
    if not text:
        return []
    matches = IDENT_RE.findall(text)  
    tokens = []
    for m in matches:
        tokens.extend(split_identifier_tokens(m))
    return tokens

def parse_diff_ids(diff_text: str) -> Tuple[Set[str], List[Tuple[str, int]]]:
    if diff_text is None:
        diff_text = ""
    lines = diff_text.splitlines()
    changed = [l[1:] for l in lines if l.startswith('+') or l.startswith('-')]  
    header = [l for l in lines[:6] if l.startswith('+++') or l.startswith('---') or l.startswith('diff ')]  # First few lines
    blob = " ".join(changed + header) 
    tokens = extract_identifiers(blob) 
    counter = Counter(tokens)  
    top = counter.most_common(10)  
    return set(counter.keys()), top

def jaccard_sets(a: Set[str], b: Set[str]) -> float:
    if not a or not b:
        return 0.0
    inter = a & b  
    uni = a | b  
    return len(inter) / len(uni) if uni else 0.0  

def build_template_from_ids(diff_text: str) -> str:
    ids_set, top_list = parse_diff_ids(diff_text)
    if not top_list: 
        return "Fix bug"
    top_token = top_list[0][0]  
    second = top_list[1][0] if len(top_list) > 1 and top_list[1][0] != top_token else None
    if second:
        return f"Fix issue in {top_token} (related to {second})"
    return f"Fix bug in {top_token}"

def rectify_by_identifiers(diff_text: str, dev_msg: str, llm_msg: str, replace_threshold: float = 0.25) -> Dict:
    diff_ids_set, _ = parse_diff_ids(diff_text)
    dev_ids = set(extract_identifiers(dev_msg))
    llm_ids = set(extract_identifiers(llm_msg))
    templ = build_template_from_ids(diff_text)
    templ_ids = set(extract_identifiers(templ))

    dev_score = jaccard_sets(diff_ids_set, dev_ids)
    llm_score = jaccard_sets(diff_ids_set, llm_ids)
    templ_score = jaccard_sets(diff_ids_set, templ_ids)

    chosen = dev_msg
    chosen_type = "dev"
    if (templ_score > max(dev_score, llm_score)) and (templ_score >= replace_threshold):
        chosen = templ
        chosen_type = "template"

    return {
        'rect_msg': chosen,
        'dev_score': dev_score,
        'llm_score': llm_score,
        'templ_score': templ_score,
        'chosen_type': chosen_type,
        'diff_ids': sorted(list(diff_ids_set))[:12],
        'templ': templ
    }

def compute_hit_rates_identifier_only(df: pd.DataFrame,
                                      hit_threshold: float = 0.2,
                                      replace_threshold: float = 0.25,
                                      drop_empty_diff: bool = True) -> Dict:
    in_df = df.copy().reset_index(drop=True)
    results = []
    for idx, row in in_df.iterrows():
        diff_text = str(row.get('Diff', '') or '')
        dev_msg = str(row.get('Message', '') or '')
        llm_msg = str(row.get('LLM Inference', '') or '')
        res = rectify_by_identifiers(diff_text, dev_msg, llm_msg, replace_threshold=replace_threshold)
        results.append(res)

    res_df = pd.DataFrame(results)
    out_df = pd.concat([in_df, res_df], axis=1)

    rect_scores = []
    for _, row in out_df.iterrows():
        diff_ids_set, _ = parse_diff_ids(str(row.get('Diff', '') or ''))
        rect_ids = set(extract_identifiers(str(row.get('rect_msg', '') or '')))
        rect_scores.append(jaccard_sets(diff_ids_set, rect_ids))
    out_df['rect_score'] = rect_scores

    eval_mask = out_df['diff_ids'].apply(lambda x: bool(x))  
    if drop_empty_diff:
        evaluable_df = out_df[eval_mask].reset_index(drop=True)
    else:
        evaluable_df = out_df.copy()

    n_total = len(out_df)
    n_evaluable = len(evaluable_df)

    dev_hits = (evaluable_df['dev_score'] >= hit_threshold).sum() if n_evaluable else 0
    llm_hits = (evaluable_df['llm_score'] >= hit_threshold).sum() if n_evaluable else 0
    rect_hits = (evaluable_df['rect_score'] >= hit_threshold).sum() if n_evaluable else 0

    dev_rate = float(dev_hits) / n_evaluable if n_evaluable else 0.0
    llm_rate = float(llm_hits) / n_evaluable if n_evaluable else 0.0
    rect_rate = float(rect_hits) / n_evaluable if n_evaluable else 0.0

    return {
        'n_total': n_total,
        'n_evaluable': n_evaluable,
        'dev_hit_rate': dev_rate,
        'llm_hit_rate': llm_rate,
        'rect_hit_rate': rect_rate,
        'df': out_df
    }

df = curr
res = compute_hit_rates_identifier_only(df, hit_threshold=0.2, replace_threshold=0.25)
print("n:", res['n_total'])
print("dev hit:", res['dev_hit_rate'])
print("llm hit:", res['llm_hit_rate'])
print("rect hit:", res['rect_hit_rate'])

n: 1000
dev hit: 0.0511482254697286
llm hit: 0.15553235908141963
rect hit: 0.1722338204592902


In [10]:
res["df"].columns = [['Hash', 'Message', 'Hashes of parents', 'Is a merge commit?',
       'Filename', 'Source Code (before)', 'Source Code (current)', 'Diff',
       'LLM Inference', 'Rectified Message']]

In [9]:
res["df"] = res["df"][['Hash', 'Message', 'Hashes of parents', 'Is a merge commit?',
       'Filename', 'Source Code (before)', 'Source Code (current)', 'Diff',
       'LLM Inference', 'rect_msg']]

In [11]:
res["df"]

,Hash,Message,Hashes of parents,Is a merge commit?,Filename,Source Code (before),Source Code (current),Diff,LLM Inference,Rectified Message
0,d2fdcba29db2a88056cfa87d9abfbdde840bd8c2,fix test name (#22),21b10ffb130aeac3131899e551939338b0dcebef,No,test_openai.py,"""""""Test OpenAI API wrapper.""""""\n\nfrom langcha...","""""""Test OpenAI API wrapper.""""""\n\nfrom langcha...","@@ -3,8 +3,8 @@\n from langchain.llms.openai i...",add missing docstring,fix test name (#22)
1,6a3dca888ba75e219185c54cd0f0ba72d1f90a26,Fix cohere integration (#33)\n\nCurrently the ...,c7f9c62532f8137db2c4ab80335ac1472e79d56a,No,cohere.py,"""""""Wrapper around Cohere APIs.""""""\nimport os\n...","""""""Wrapper around Cohere APIs.""""""\nimport os\n...","@@ -22,7 +22,7 @@ class Cohere(BaseModel, LLM)...",model attribute for gptd-instruct-tft,Fix cohere integration (#33)\n\nCurrently the ...
2,90a6e578bc8d2147f5fcc22d6413686027eca2b5,fix type hint (#34),6a3dca888ba75e219185c54cd0f0ba72d1f90a26,No,cohere.py,"""""""Wrapper around Cohere APIs.""""""\nimport os\n...","""""""Wrapper around Cohere APIs.""""""\nimport os\n...","@@ -22,7 +22,7 @@ class Cohere(BaseModel, LLM)...",model attribute for model_name_to_json,Fix issue in model (related to str)
3,eb36317f9ae6054a9bbf62ac7ceb8bae388ee811,Harrison/fix imports (#72)\n\nfix imports and ...,a5b61d59e193c4f9804c9dece7bd1a1c5c1103e0,No,embeddings.ipynb,"{\n ""cells"": [\n {\n ""cell_type"": ""code"",\n...","{\n ""cells"": [\n {\n ""cell_type"": ""code"",\n...","@@ -5,11 +5,26 @@\n ""execution_count"": 1,\n...",update error message,Harrison/fix imports (#72)\n\nfix imports and ...
4,eb36317f9ae6054a9bbf62ac7ceb8bae388ee811,Harrison/fix imports (#72)\n\nfix imports and ...,a5b61d59e193c4f9804c9dece7bd1a1c5c1103e0,No,mrkl.ipynb,"{\n ""cells"": [\n {\n ""cell_type"": ""code"",\n...","{\n ""cells"": [\n {\n ""cell_type"": ""markdown...","@@ -1,5 +1,14 @@\n {\n ""cells"": [\n+ {\n+ ...",add a notebook to the default cell_type,Harrison/fix imports (#72)\n\nfix imports and ...
...,...,...,...,...,...,...,...,...,...,...
995,08876ad0660222fcc481175639c59ce9f2b110bb,"Fix SelfQueryRetriever, passing new query to v...",8fd4d5d11752ad268f8af90f39e73742d89197fc,No,base.py,"""""""Retriever that generates and executes struc...","""""""Retriever that generates and executes struc...","@@ -81,7 +81,7 @@ class SelfQueryRetriever(Bas...",fix bug in vectorstore search,"Fix SelfQueryRetriever, passing new query to v..."
996,11341fcecbb92ccdb8ee2edf72c83342ed59f6fd,Fixed query checker for SQLDatabaseChain (#478...,08876ad0660222fcc481175639c59ce9f2b110bb,No,base.py,"""""""Chain for interacting with SQL Database.""""""...","""""""Chain for interacting with SQL Database.""""""...","@@ -130,7 +130,7 @@ class SQLDatabaseChain(Cha...",fix llm prompt,Fixed query checker for SQLDatabaseChain (#478...
997,e28f4a5f391bc8c90079c26d3ba92256e1d589a9,changed cohere.py to update the default model ...,75fe9d3555d118e21650445ef1a390e8726ced00,No,cohere.py,"""""""Wrapper around Cohere embedding models.""""""\...","""""""Wrapper around Cohere embedding models.""""""\...","@@ -18,11 +18,13 @@ class CohereEmbeddings(Bas...",fix typo in example,changed cohere.py to update the default model ...
998,432421ffa50c44e660b902adb48db40add9621d5,[Fix][GenerativeAgent] Get the memory importan...,be405ac1398abd9335b93bf7567e02f0381d5811,No,memory.py,import logging\nimport re\nfrom datetime impor...,import logging\nimport re\nfrom datetime impor...,"@@ -123,7 +123,7 @@ class GenerativeAgentMemor...",fix importance,[Fix][GenerativeAgent] Get the memory importan...


In [13]:
res["df"].to_csv("Lab2final.csv", index=False)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

OUT_DIR = "report_assets"
os.makedirs(OUT_DIR, exist_ok=True)

def save_summary_table(res, out_path=os.path.join(OUT_DIR, "summary_table.tex")):
    n_total = res.get('n_total', len(res['df']))
    n_eval = res.get('n_evaluable', len(res['df']))
    dev = res['dev_hit_rate']
    llm = res['llm_hit_rate']
    rect = res['rect_hit_rate']
    replacement_count = (res['df']['chosen_type'] == 'template').sum()
    replacement_rate = replacement_count / n_eval if n_eval else 0.0

    summary = pd.DataFrame([{
        "n_total": n_total,
        "n_evaluable": n_eval,
        "dev_hit": dev,
        "llm_hit": llm,
        "rect_hit": rect,
        "replacement_count": replacement_count,
        "replacement_rate": replacement_rate
    }])
    latex = summary.to_latex(index=False, float_format="%.3f", caption="Summary hit rates and replacements", label="tab:summary", column_format='lrrrrrr', longtable=False, header=True)
    with open(out_path, "w") as f:
        f.write(latex)
    return summary

def plot_hit_rates_bar(res, out_png=os.path.join(OUT_DIR, "hit_rates_bar.png")):
    vals = [res['dev_hit_rate'], res['llm_hit_rate'], res['rect_hit_rate']]
    labels = ['Developer', 'LLM', 'Rectifier']
    fig, ax = plt.subplots(figsize=(6, 4))
    bars = ax.bar(labels, vals, edgecolor='k')
    ax.set_ylim(0, max(0.6, max(vals) * 1.15))
    ax.set_ylabel("Hit rate")
    ax.set_title("Hit rates (per-file, evaluable samples)")
    for bar, val in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width() / 2, val + 0.01, f"{val:.3f}", ha='center')
    plt.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)

def plot_rect_score_hist(df, out_png=os.path.join(OUT_DIR, "rect_score_hist.png")):
    vals = df['rect_score'].dropna()
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.hist(vals, bins=20, edgecolor='k')
    ax.set_title("Distribution of rectifier Jaccard scores")
    ax.set_xlabel("rect_score")
    ax.set_ylabel("count")
    plt.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)

def plot_replacement_breakdown(df, out_png=os.path.join(OUT_DIR, "replacement_breakdown_bar.png")):
    counts = df['chosen_type'].value_counts()
    fig, ax = plt.subplots(figsize=(6, 4))
    counts.plot(kind='bar', ax=ax)
    ax.set_title("Rectifier chosen type distribution")
    ax.set_ylabel("count")
    for i, v in enumerate(counts.values):
        ax.text(i, v + max(counts.values) * 0.01, str(v), ha='center')
    plt.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)

def save_examples_table(df, out_tex=os.path.join(OUT_DIR, "top_examples.tex"), n=10):
    replaced = df[df['chosen_type'] == 'template']
    top = replaced.sort_values('rect_score', ascending=False).head(n // 2)
    bot = replaced.sort_values('rect_score', ascending=True).head(n // 2)
    ex = pd.concat([top, bot])
    ex_table = ex[['Diff', 'Message', 'LLM Inference', 'rect_msg', 'dev_score', 'llm_score', 'templ_score', 'rect_score', 'chosen_type']].copy()
    ex_table['Diff'] = ex_table['Diff'].str.replace(r'\s+', ' ', regex=True).str.slice(0, 120)
    ex_table[['Message', 'LLM Inference', 'rect_msg']] = ex_table[['Message', 'LLM Inference', 'rect_msg']].applymap(
        lambda s: (s[:80] + '...') if isinstance(s, str) and len(s) > 80 else s)
    latex = ex_table.to_latex(index=False, escape=True, column_format='p{3cm}p{2.2cm}p{2.2cm}p{2.2cm}rrrrc', caption="Representative rectifier examples", label="tab:examples")
    with open(out_tex, "w") as f:
        f.write(latex)
    return ex_table

def sensitivity_curve(df, thresholds=np.linspace(0.05, 0.6, 12), hit_thr=0.2, out_png=os.path.join(OUT_DIR, "sensitivity_curve.png")):
    rect_rates = []
    n_eval = (df['diff_ids'].apply(bool)).sum()
    for t in thresholds:
        templ_better = (df['templ_score'] > df['dev_score']) & (df['templ_score'] > df['llm_score']) & (df['templ_score'] >= t)
        rect_score = np.where(templ_better, df['templ_score'], df['dev_score'])
        mask = df['diff_ids'].apply(bool)
        rect_hit = ((rect_score >= hit_thr) & mask).sum()
        rect_rates.append(rect_hit / mask.sum() if mask.sum() else 0.0)
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(thresholds, rect_rates, marker='o')
    ax.set_xlabel("replace_threshold (templ_score)")
    ax.set_ylabel("Rectifier hit rate")
    ax.set_title("Sensitivity of rectifier to replace_threshold")
    plt.grid(True)
    plt.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)
    return thresholds, rect_rates

summary = save_summary_table(res)
plot_hit_rates_bar(res)
plot_rect_score_hist(res['df'])
plot_replacement_breakdown(res['df'])
examples = save_examples_table(res['df'], n=10)
s_thresh, s_rates = sensitivity_curve(res['df'])

pd.DataFrame({"threshold": s_thresh, "rect_rate": s_rates}).to_csv(os.path.join(OUT_DIR, "sensitivity.csv"), index=False)

print("Saved assets to", OUT_DIR)

Saved assets to report_assets
McNemar: {'b': 142, 'c': 126, 'chi2': 0.8395522388059702, 'p': 0.35952486213934653}


C:\Users\Surrya Gokul\AppData\Local\Temp\ipykernel_43700\4008949819.py:87: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ex_table[['Message', 'LLM Inference', 'rect_msg']] = ex_table[['Message', 'LLM Inference', 'rect_msg']].applymap(
